# Construir banco de dados vetorial
Obtêm dados do JSON de produtos e efetua o push pro database

## Preparar ambiente

In [1]:
# Instalar dependências
#!pip install pinecone

In [2]:
from pinecone import Pinecone, ServerlessSpec
import openai
import pandas as pd
import os
import dotenv
from sentence_transformers import SentenceTransformer # para embeddings

c:\Users\aleju\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Variáveis de ambiente

In [3]:
dotenv.load_dotenv()

True

In [4]:
token = os.getenv("OPENROUTER_API_KEY") # token do OpenRouter
model_name = os.getenv("MODEL_NAME") # nome do modelo

pinecone_api_key = os.getenv("PINECODE_API_KEY") # token do Pinecone
pinecode_index_name = os.getenv("PINECODE_INDEX_NAME")  # nome do índice do Pinecone

# Modelo de embedding
embedding_model = os.getenv("EMBEDDING_MODEL_NAME")
# Carregar modelo localmente
embedding_client = SentenceTransformer(embedding_model)

### Iniciar Pinecode

In [5]:
pc = Pinecone(api_key=pinecone_api_key)

# client = OpenAI(api_key=token, base_url=open_ai_base_url)

# Teste de embeddings

Gerar **embeddings vetoriais** (transformar texto em vetores de números):

In [21]:
# Se usar RunPod: 
# output = client.embeddings.create(input=["Hello World!"], model=model_name)
# embedding = output.data[0].embedding

# Chama o modelo de embedding para transformar o texto em um valor numérico
output = embedding_client.encode(
    sentences=["Hello world"], # Lista de sentenças que serão codificadas
    convert_to_numpy=True # Converte a saída para um array do NumPy
)
# Como o sentence possui apenas 1 string, o output será um array 1D (1, 384), então acessamos o primeiro (e único) elemento do array
embedding = output[0]

print(embedding)
print("\nTamanho do vetor embedding: ", len(embedding))

[ 1.51960850e-02 -2.25706734e-02  8.54711980e-03 -7.41705820e-02
  3.83639405e-03  2.71344697e-03 -3.12679186e-02  4.46339697e-02
  4.40551378e-02 -7.87114259e-03 -2.52007879e-02 -3.33665088e-02
  1.44279227e-02  4.65381928e-02  8.55509657e-03 -1.61457192e-02
  7.40579702e-03 -1.90124381e-02 -1.14726298e-01 -1.81576442e-02
  1.26359358e-01  2.97029372e-02  2.52810232e-02 -3.42178792e-02
 -4.09996919e-02  6.61730114e-03  1.02706430e-02  2.23622601e-02
  4.43633646e-03 -1.27309680e-01 -1.61492061e-02 -2.03801412e-02
  4.72120903e-02  1.15798796e-02  6.81871623e-02  7.29864091e-03
 -1.78529993e-02  4.07821722e-02 -1.02694668e-02  2.37570591e-02
  1.06028914e-02 -2.85843518e-02  8.15969706e-03 -1.51805691e-02
  3.08962502e-02 -6.59799501e-02 -2.21965052e-02  5.40237539e-02
  2.54224031e-03  2.24527121e-02 -9.16537493e-02 -4.51403186e-02
 -4.19208128e-03 -5.62152732e-03 -5.38092293e-03  9.83934850e-02
  6.05247915e-02  7.42288399e-03  1.39386216e-02  2.68772594e-03
  4.75693829e-02  2.86365